In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.5)

In [ ]:
#Import data
url = '2016_Green_Taxi_Trip_Data.csv'
dftaxi = pd.read_csv(url)

In [ ]:
#%% Ensure data quality. Do not have incorrect longitudes's in dataset. 
dftaxi[dftaxi['Pickup_longitude'] > 0]   

In [ ]:
#%% QC to check the differences between latitudes/longitudes and response variable. 7M row discrepancy. 
dftaxi.count()

In [ ]:
#%% Parse date from datetime
dftaxi['pickup_timestring'] = dftaxi['lpep_pickup_datetime'].apply(lambda x: x.split(" ")[0]) #pickup_timestring
dftaxi['dropoff_timestring'] = dftaxi['Lpep_dropoff_datetime'].apply(lambda x: x.split(" ")[0]) #dropoff_timestring
#%% Assign Response variable
dftaxi['response_variable'] = np.where(dftaxi.pickup_timestring is not None & dftaxi.dropoff_timestring is not None, '1',0)
dftaxi.head()

In [ ]:
#%% Check to ensure that all rows have a response variable
dftaxi[dftaxi['response_variable'] == '0'] 
#returns empty dataframe! 

In [ ]:
#%% Get month out of timestring
dftaxi['pickup_timestring_month'] = dftaxi['pickup_timestring'].apply(lambda x: x.split("/")[0]) #pickup_timestring_month
print(dftaxi.head())
#%% Get Day out of timestring
dftaxi['pickup_timestring_day'] = dftaxi['pickup_timestring'].apply(lambda x: x.split("/")[1]) #pickup_timestring_day

In [ ]:
#%% Subset to relevant columns.
dftaxi = dftaxi[['pickup_timestring','pickup_timestring_month','pickup_timestring_day','Passenger_count','Trip_distance','Total_amount','response_variable']] # Add response variable
print(dftaxi.head())
print(dftaxi.dtypes)

In [ ]:
#%% Convert to numeric
#dftaxi['Total_amount']=dftaxi['Total_amount'].apply(pd.to_numeric)
dftaxi[['Total_amount','response_variable']] = dftaxi[['Total_amount','response_variable']].convert_objects(convert_numeric=True) #deprecated, yet works
print(dftaxi.dtypes)

In [ ]:
#%% Group by day
dftaxi_day = dftaxi[['pickup_timestring','pickup_timestring_month','pickup_timestring_day','Passenger_count','Trip_distance','Total_amount','response_variable']]
dftaxi_day = dftaxi_day.groupby(['pickup_timestring','pickup_timestring_month','pickup_timestring_day'],as_index=False).sum()
print(dftaxi_day.head())
print('Number of rows in daily aggregated view', len(dftaxi_day))

In [ ]:
#%% Export clean file to CSV
dftaxi_day.to_csv('dftaxi_by_day.csv',sep=',',index=False,header=True)

In [ ]:
dftaxi_day.head()
#%% Patterns in daily data 
dftaxi_day.plot(x='pickup_timestring_month', y='response_variable',kind='line')